In [ ]:
# Set credentials
import os

os.environ["MLFLOW_TRACKING_URI"] = "http://mlflow.mlnow.frenoid.com:30080/"
os.environ["MLFLOW_TRACKING_USERNAME"] = "<>"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "<>"

In [ ]:
# This uses the mlflow.<model_flavour>.log_model() method to log a model to the registry
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature


# This method works best using env to get credentials
with mlflow.start_run() as run:
    X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    params = {"max_depth": 2, "random_state": 42}
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)

    # Infer the model signature
    y_pred = model.predict(X_test)
    signature = infer_signature(X_test, y_pred)

    # Log parameters and metrics using the MLflow APIs
    mlflow.log_params(params)
    mlflow.log_metrics({"mse": mean_squared_error(y_test, y_pred)})

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name="sk-learn-random-forest-reg-model",
    )

In [ ]:
# This demonstartes how to set and delete aliases on models
from mlflow import MlflowClient

client = MlflowClient()

# Create "Champion" alias for version 1 of model "sk-learn-random-forest-reg-model"
client.set_registered_model_alias("sk-learn-random-forest-reg-model", "Champion", "1")

# Reassign the champion alias to version 2
client.set_registered_model_alias("sk-learn-random-forest-reg-model", "Champion", "2")

# Get model version by alias
champion_model = client.get_model_version_by_alias("sk-learn-random-forest-reg-model", "Champion")
print(champion_model)

# Delete the alias
client.delete_registered_model_alias("sk-learn-random-forest-reg-model", "Champion")

In [ ]:
# This demonstrates how to set and delete tags using the MLflow Client API
from mlflow import MlflowClient

client = MlflowClient()

# Set registered model tag
client.set_registered_model_tag("sk-learn-random-forest-reg-model", "task", "classification")

# Get registered models with the task: classification tag
classification_models = client.search_registered_models(
    filter_string="tags.`task` = 'classification'"
)
print(classification_models)

# Delete the registered model tag
client.delete_registered_model_tag("sk-learn-random-forest-reg-model", "task")

# Check the search no longer returns any registered models
classification_models = client.search_registered_models(
    filter_string="tags.`task` = 'classification'"
)
print(classification_models)

# Set model version tag
client.set_model_version_tag("sk-learn-random-forest-reg-model", "1", "validation_status", "approved")

# Get model versions with validation_status: approved tag
approved_models = client.search_model_versions(
     filter_string="tags.`validation_status` = 'approved'"   
)
print(approved_models)

In [ ]:
# This demonstrates how to fetch a specific model version
# You simply supply the version number as part of the model URI
import mlflow.pyfunc

model_name = "sk-learn-random-forest-reg-model"
model_version = 1

# Get the model object
from mlflow import MlflowClient
mv1 = MlflowClient().get_model_version(model_name, model_version)
print(mv1.source)

# Using mlflow-artifacts: cause the client to download from the tracking server
model = mlflow.pyfunc.load_model(model_uri=mv1.source)
print(model)

In [ ]:
# Use the model to make predictions
from sklearn.datasets import make_regression
data, _ = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)

model.predict(data)